<div align="center", style="color: purple">

# Zadanie 1

</div>

Importy

In [4]:
import numpy as np
import copy

Funkcje potrzebne do napisania ES

In [1]:
# Tworzy losowego osobnika
def random_individual(n: int, x_min: float = -5.0, x_max: float = 5.0):
    return {
        'x': np.random.uniform(x_min, x_max, size=n),
        'o': np.random.uniform(0.1, 1.0, size=n)   # sigma
    }

# Tworzy populację
def random_population(mu: int, n: int, x_min: float = -5.0, x_max: float = 5.0):
    return np.array([random_individual(n, x_min, x_max) for _ in range(mu)], dtype=object)

# Ocena populacji
def population_evaluation(P, F):
    return np.array([F(ind['x']) for ind in P])

# Selekcja rodziców (z powtórzeniami!)
def parent_selection(P, lam: int):
    idx = np.random.choice(len(P), lam, replace=True)
    return P[idx]

# Mutacja gaussowska
def mutation(Pc, tau, tau0, low, high):
    lam = len(Pc)
    n = len(Pc[0]['x'])

    for i in range(lam):
        indiv = Pc[i]
        x = indiv['x']
        o = indiv['o']

        # Mutacja parametrów strategii
        N0 = np.random.normal()
        Ni = np.random.normal(size=n)

        o_new = o * np.exp(tau0 * N0 + tau * Ni)

        # Mutacja wartości x
        x_new = x + o_new * np.random.normal(size=n)
        x_new = np.clip(x_new, low, high)

        indiv['x'] = x_new
        indiv['o'] = o_new

    return Pc

def replacement_plus(P, Pc, mu, F):
    combined = np.array([copy.deepcopy(ind) for ind in list(P) + list(Pc)], dtype=object)
    fitness = population_evaluation(combined, F)
    idx = np.argsort(fitness)[-mu:]   # najlepsze mu
    return combined[idx]

# Replacement: ES(mu, lambda)
def replacement_comma(Pc, mu, F):
    combined = np.array([copy.deepcopy(ind) for ind in list(Pc)], dtype=object)
    fitness = population_evaluation(combined, F)
    idx = np.argsort(fitness)[-mu:]   # najlepsze mu
    return combined[idx]

### ES

In [2]:
ITER = 10000

# Strategia ewolucyjna ES(mu + lambda)
def ES_plus(mu, lam, tau, tau0, F, n, ITER=ITER, x_min: float = -5.0, x_max: float = 5.0):
    P = random_population(mu, n, x_min, x_max)

    for _ in range(ITER):
        Pc = parent_selection(P, lam)
        Pc = mutation(Pc, tau, tau0, x_min, x_max)
        P = replacement_plus(P, Pc, mu, F)

    fitness = population_evaluation(P, F)
    return P[np.argmax(fitness)]

# Strategia ewolucyjna ES(mu, lambda)
def ES_comma(mu, lam, tau, tau0, F, n, ITER=ITER, x_min: float = -5.0, x_max: float = 5.0):
    P = random_population(mu, n, x_min, x_max)

    for _ in range(ITER):
        Pc = parent_selection(P, lam)
        Pc = mutation(Pc, tau, tau0, x_min, x_max)
        P = replacement_comma(Pc, mu, F)

    fitness = population_evaluation(P, F)
    return P[np.argmax(fitness)]

## Problemy bez ograniczeń

In [13]:
def sphere(x):
    return -np.sum(x**2)  # minimalizacja -> dajemy minus

D = 100
TAU = lambda K: K / np.sqrt(2 * D)
TAU0 = lambda K: K / np.sqrt(2 * np.sqrt(D))

params = [
    # 1. Konfiguracja bazowa – mała populacja, umiarkowana mutacja
    {
        'mu': 10,
        'lam': 30,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 2. Większa liczba potomków, mocniejsza mutacja – większa eksploracja
    {
        'mu': 10,
        'lam': 70,
        'tau': TAU(0.2),
        'tau0': TAU0(0.2),
    },
    # 3. Duża populacja rodziców i potomków, umiarkowana mutacja – bardziej „stabilne” szukanie
    {
        'mu': 30,
        'lam': 200,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 4. Mała populacja, słaba mutacja – bardziej eksploatacja lokalna
    {
        'mu': 5,
        'lam': 25,
        'tau': TAU(0.05),
        'tau0': TAU0(0.05),
    },
]

for p in params:
    # ES(μ + λ)
    best_plus = ES_plus(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=sphere,
        n=D,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ+λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_plus}\n"
        f"Wartość, którą osiąga: {-sphere(best_plus)}\n"
    )

    # ES(μ, λ)
    best_comma = ES_comma(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=sphere,
        n=D,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ,λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_comma}\n"
        f"Wartość, którą osiąga: {-sphere(best_comma)}\n"
    )

[ES(μ+λ)] mu=10, λ=30
Najlepszy osobnik: [-2.63940376e-03 -2.04109342e-02 -1.76138562e-02  1.79646233e+00
  3.67588887e-03 -2.18527160e-04 -1.19779721e-03  1.15776263e-02
 -2.92859606e-03  1.99867615e-03 -9.85845381e-03  1.83303839e-02
  1.16135798e-02 -4.32198488e-02 -7.92431598e-03 -1.27468353e-02
  2.58544155e-01  3.68435771e-03  5.09881211e-03 -3.40437691e-03
  2.12043297e-02  4.23893740e-03  5.92269042e-03 -2.05366296e-02
 -1.24306995e-03 -1.30979575e-02 -2.68087054e-03 -1.94828919e-03
  2.78826926e-04  6.00614069e-03 -8.47774803e-03  1.15335338e-03
 -1.87821639e-02  2.01922551e-03  2.27107265e-03 -5.89818504e-03
  2.45891078e-02 -9.09229748e-02 -1.56121888e-02 -4.53153375e+00
  3.23739215e-03 -1.43817141e-02 -1.96900871e-02 -8.02456098e-03
  1.28458031e-04  1.50683008e-02  1.21953330e-02  5.36055401e-03
 -1.45139841e-02 -1.75210423e-02  7.95549670e-03  3.77875623e-03
  5.44013695e-04 -1.46979972e-02 -3.11773596e-02 -7.54924788e-03
  1.16376005e-03  1.31212907e-02 -1.06637816e-03 

In [14]:
def easom(x):
    x1, x2 = x[0], x[1]
    return np.cos(x1) * np.cos(x2) * np.exp(-((x1 - np.pi)**2 + (x2 - np.pi)**2))

D = 2
TAU = lambda K: K / np.sqrt(2 * D)
TAU0 = lambda K: K / np.sqrt(2 * np.sqrt(D))

params = [
    # 1. Konfiguracja bazowa – mała populacja, umiarkowana mutacja
    {
        'mu': 10,
        'lam': 30,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 2. Większa liczba potomków, mocniejsza mutacja – większa eksploracja
    {
        'mu': 10,
        'lam': 70,
        'tau': TAU(0.2),
        'tau0': TAU0(0.2),
    },
    # 3. Duża populacja rodziców i potomków, umiarkowana mutacja – bardziej „stabilne” szukanie
    {
        'mu': 30,
        'lam': 200,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 4. Mała populacja, słaba mutacja – bardziej eksploatacja lokalna
    {
        'mu': 5,
        'lam': 25,
        'tau': TAU(0.05),
        'tau0': TAU0(0.05),
    },
]

for p in params:
    # ES(μ + λ)
    best_plus = ES_plus(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=easom,
        n=D,
        x_min=-100.0,
        x_max=100.0
    )['x']
    print(
        f"[ES(μ+λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_plus}\n"
        f"Wartość, którą osiąga: {-easom(best_plus)}\n"
    )

    # ES(μ, λ)
    best_comma = ES_comma(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=easom,
        n=D,
        x_min=-100.0,
        x_max=100.0
    )['x']
    print(
        f"[ES(μ,λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_comma}\n"
        f"Wartość, którą osiąga: {-easom(best_comma)}\n"
    )

[ES(μ+λ)] mu=10, λ=30
Najlepszy osobnik: [3.14159265 3.14159265]
Wartość, którą osiąga: -1.0

[ES(μ,λ)] mu=10, λ=30
Najlepszy osobnik: [ 3.14159267 14.18242247]
Wartość, którą osiąga: -5.1887108789118556e-55

[ES(μ+λ)] mu=10, λ=70
Najlepszy osobnik: [3.14159266 3.14159266]
Wartość, którą osiąga: -1.0

[ES(μ,λ)] mu=10, λ=70
Najlepszy osobnik: [3.14159265 3.14159266]
Wartość, którą osiąga: -1.0

[ES(μ+λ)] mu=30, λ=200
Najlepszy osobnik: [3.14159265 3.14159265]
Wartość, którą osiąga: -1.0

[ES(μ,λ)] mu=30, λ=200
Najlepszy osobnik: [3.14159266 3.14159265]
Wartość, którą osiąga: -1.0

[ES(μ+λ)] mu=5, λ=25
Najlepszy osobnik: [-62.46766684 -21.44389862]
Wartość, którą osiąga: 0.0

[ES(μ,λ)] mu=5, λ=25
Najlepszy osobnik: [3.14159266 3.14159265]
Wartość, którą osiąga: -1.0



In [15]:
def dixon_price(x):
    x = np.array(x)
    term1 = (x[0] - 1)**2
    term2 = np.sum(np.arange(2, len(x)+1) * (2*x[1:]**2 - x[:-1])**2)
    return -(term1 + term2)

D = 100
TAU = lambda K: K / np.sqrt(2 * D)
TAU0 = lambda K: K / np.sqrt(2 * np.sqrt(D))

params = [
    # 1. Konfiguracja bazowa – mała populacja, umiarkowana mutacja
    {
        'mu': 10,
        'lam': 30,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 2. Większa liczba potomków, mocniejsza mutacja – większa eksploracja
    {
        'mu': 10,
        'lam': 70,
        'tau': TAU(0.2),
        'tau0': TAU0(0.2),
    },
    # 3. Duża populacja rodziców i potomków, umiarkowana mutacja – bardziej „stabilne” szukanie
    {
        'mu': 30,
        'lam': 200,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 4. Mała populacja, słaba mutacja – bardziej eksploatacja lokalna
    {
        'mu': 5,
        'lam': 25,
        'tau': TAU(0.05),
        'tau0': TAU0(0.05),
    },
]

for p in params:
    # ES(μ + λ)
    best_plus = ES_plus(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=dixon_price,
        n=D,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ+λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_plus}\n"
        f"Wartość, którą osiąga: {-dixon_price(best_plus)}\n"
    )

    # ES(μ, λ)
    best_comma = ES_comma(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=dixon_price,
        n=D,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ,λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_comma}\n"
        f"Wartość, którą osiąga: {-dixon_price(best_comma)}\n"
    )

[ES(μ+λ)] mu=10, λ=30
Najlepszy osobnik: [-5.51675544  0.92118     0.61316757 -0.31527112 -0.08154114  0.07697016
 -0.26388533 -0.71541241  0.17201141  0.55911531  0.66755789 -0.85754702
 -0.19429704 -0.01728548  0.02402296  0.19713005  0.36674175  0.46254342
  0.47327114 -0.42135209  0.08111667  0.3018323   0.4624532   0.5337297
 -0.29785562 -0.10602676  0.0669573   0.00741077 -0.03799291  0.06456205
  0.18743873  0.3051313  -0.38170336  0.11959922  0.37773192  0.51803149
  0.68417532  0.61295698  0.52970601  0.45354424  0.32791293  0.14656497
 -0.04764352  0.15828453  0.53548328  0.83144687  1.33062365 -0.79060269
  0.08118647  0.50503922  0.76127836  0.63127999  0.57749161  0.47840951
  0.37044107  0.26993051  0.14514344 -0.00674339  0.01074713  0.07206083
  0.20796899  0.3196298   0.44904387  0.52160599  0.52636473 -0.36232848
 -0.00687327 -0.00920941  0.08852719  0.21410895  0.32211841  0.35408126
  0.32679292  0.27045441  0.52228602 -0.73104166  0.04320687 -0.22865643
  0.0206668

In [16]:
def rastrigin(x):
    x = np.array(x)
    n = len(x)
    return -(10 * n + np.sum(x**2 - 10 * np.cos(2 * np.pi * x)))

D = 100
TAU = lambda K: K / np.sqrt(2 * D)
TAU0 = lambda K: K / np.sqrt(2 * np.sqrt(D))

params = [
    # 1. Konfiguracja bazowa – mała populacja, umiarkowana mutacja
    {
        'mu': 10,
        'lam': 30,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 2. Większa liczba potomków, mocniejsza mutacja – większa eksploracja
    {
        'mu': 10,
        'lam': 70,
        'tau': TAU(0.2),
        'tau0': TAU0(0.2),
    },
    # 3. Duża populacja rodziców i potomków, umiarkowana mutacja – bardziej „stabilne” szukanie
    {
        'mu': 30,
        'lam': 200,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 4. Mała populacja, słaba mutacja – bardziej eksploatacja lokalna
    {
        'mu': 5,
        'lam': 25,
        'tau': TAU(0.05),
        'tau0': TAU0(0.05),
    },
]

for p in params:
    # ES(μ + λ)
    best_plus = ES_plus(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=rastrigin,
        n=D,
        x_min=-5.12,
        x_max=5.12
    )['x']
    print(
        f"[ES(μ+λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_plus}\n"
        f"Wartość, którą osiąga: {-rastrigin(best_plus)}\n"
    )

    # ES(μ, λ)
    best_comma = ES_comma(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=rastrigin,
        n=D,
        x_min=-5.12,
        x_max=5.12
    )['x']
    print(
        f"[ES(μ,λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_comma}\n"
        f"Wartość, którą osiąga: {-rastrigin(best_comma)}\n"
    )

[ES(μ+λ)] mu=10, λ=30
Najlepszy osobnik: [ 3.98017283e+00 -1.61773774e-04  1.98973446e+00 -9.94826472e-01
  9.94794078e-01  3.97989062e+00  1.98982771e+00 -9.95130076e-01
 -1.98997773e+00  9.95015920e-01 -1.99032236e+00  1.98971300e+00
  2.80662550e-05  2.98487427e+00  2.98444438e+00 -9.94672038e-01
  4.97636197e+00  1.98986164e+00  3.98014295e+00  3.97967983e+00
 -1.98993529e+00 -2.98422331e+00  9.94807811e-01 -2.98516755e+00
 -9.96493563e-01 -2.98452381e+00  9.94952985e-01 -1.98921067e+00
  9.97068788e-01 -1.98991930e+00  9.94125637e-01  4.11008014e+00
  2.98503670e+00 -9.95127841e-01 -1.98986912e+00 -1.99031806e+00
  9.95264383e-01 -2.98546644e+00 -9.95085444e-01  9.94870302e-01
  1.99092245e+00  2.98523013e+00 -2.98394826e+00 -1.98995723e+00
 -9.94870022e-01 -2.98499879e+00  7.98845858e-04  9.94633660e-01
  9.95037184e-01  1.98908871e+00  1.99023695e+00 -9.94818221e-01
  1.98903653e+00  3.97961279e+00 -1.98980953e+00  9.94854585e-01
  1.02108128e-04  1.98981621e+00 -3.78769333e-04 

In [9]:
def schwefel(x):
    x = np.array(x)
    n = len(x)
    return -(418.9829 * n - np.sum(x * np.sin(np.sqrt(np.abs(x)))))

D = 10
TAU = lambda K: K / np.sqrt(2 * D)
TAU0 = lambda K: K / np.sqrt(2 * np.sqrt(D))

params = [
    # 1. Konfiguracja bazowa – mała populacja, umiarkowana mutacja
    {
        'mu': 1000,
        'lam': 3000,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 2. Większa liczba potomków, mocniejsza mutacja – większa eksploracja
    {
        'mu': 10,
        'lam': 70,
        'tau': TAU(0.2),
        'tau0': TAU0(0.2),
    },
    # 3. Duża populacja rodziców i potomków, umiarkowana mutacja – bardziej „stabilne” szukanie
    {
        'mu': 30,
        'lam': 200,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 4. Mała populacja, słaba mutacja – bardziej eksploatacja lokalna
    {
        'mu': 5,
        'lam': 25,
        'tau': TAU(0.05),
        'tau0': TAU0(0.05),
    },
]

for p in params:
    # ES(μ + λ)
    best_plus = ES_plus(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=schwefel,
        n=D,
        x_min=-500.0,
        x_max=500.0
    )['x']
    print(
        f"[ES(μ+λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_plus}\n"
        f"Wartość, którą osiąga: {-schwefel(best_plus)}\n"
    )

    # ES(μ, λ)
    best_comma = ES_comma(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=schwefel,
        n=D,
        x_min=-500.0,
        x_max=500.0
    )['x']
    print(
        f"[ES(μ,λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_comma}\n"
        f"Wartość, którą osiąga: {-schwefel(best_comma)}\n"
    )


[ES(μ+λ)] mu=1000, λ=3000
Najlepszy osobnik: [ 420.96874634 -302.52493568 -124.82935665   65.54786467  420.96874656
  420.96874643    5.23919913  420.96874641  420.96874626  203.81425185]
Wartość, którą osiąga: 1402.0703360072193

[ES(μ,λ)] mu=1000, λ=3000
Najlepszy osobnik: [-302.52493558   65.54786508  -25.87741687  420.96874649  420.96874637
   65.54786509  420.96874631  420.96874648  420.96874569   65.54786493]
Wartość, którą osiąga: 1579.382104902103

[ES(μ+λ)] mu=10, λ=70
Najlepszy osobnik: [ 420.96874824 -302.52493652  420.96875487  420.96874776  420.96874627
 -302.52492398  420.96875115  420.96874647   65.5478654  -124.82935582]
Wartość, którą osiąga: 888.3314155839653

[ES(μ,λ)] mu=10, λ=70
Najlepszy osobnik: [ -25.87746402  420.96874743  420.9687849   203.81425317 -302.52485472
  203.81395089 -124.82935613  420.96874687  420.96874611 -124.94952557]
Wartość, którą osiąga: 1539.8330177560583

[ES(μ+λ)] mu=30, λ=200
Najlepszy osobnik: [ 203.8142528   203.8142526  -124.82935682  

In [18]:
def griewank(x):
    x = np.array(x)
    n = len(x)

    sum_term = np.sum(x**2) / 4000
    prod_term = np.prod(np.cos(x / np.sqrt(np.arange(1, n+1))))

    return -(1 + sum_term - prod_term)

D = 100
TAU = lambda K: K / np.sqrt(2 * D)
TAU0 = lambda K: K / np.sqrt(2 * np.sqrt(D))

params = [
    # 1. Konfiguracja bazowa – mała populacja, umiarkowana mutacja
    {
        'mu': 10,
        'lam': 30,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 2. Większa liczba potomków, mocniejsza mutacja – większa eksploracja
    {
        'mu': 10,
        'lam': 70,
        'tau': TAU(0.2),
        'tau0': TAU0(0.2),
    },
    # 3. Duża populacja rodziców i potomków, umiarkowana mutacja – bardziej „stabilne” szukanie
    {
        'mu': 30,
        'lam': 200,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 4. Mała populacja, słaba mutacja – bardziej eksploatacja lokalna
    {
        'mu': 5,
        'lam': 25,
        'tau': TAU(0.05),
        'tau0': TAU0(0.05),
    },
]

for p in params:
    # ES(μ + λ)
    best_plus = ES_plus(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=griewank,
        n=D,
        x_min=-600.0,
        x_max=600.0
    )['x']
    print(
        f"[ES(μ+λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_plus}\n"
        f"Wartość, którą osiąga: {-griewank(best_plus)}\n"
    )

    # ES(μ, λ)
    best_comma = ES_comma(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=griewank,
        n=D,
        x_min=-600.0,
        x_max=600.0
    )['x']
    print(
        f"[ES(μ,λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_comma}\n"
        f"Wartość, którą osiąga: {-griewank(best_comma)}\n"
    )

[ES(μ+λ)] mu=10, λ=30
Najlepszy osobnik: [ 3.08070338e+00 -2.81898533e-02  5.80768133e-02  1.32556220e-01
  3.02151303e-02  2.46901013e-02  1.62157680e-01  5.01713614e-03
  1.93919175e-01  1.33341196e-01  1.54300526e-01 -2.62733705e+00
  2.64951993e-01 -8.65875711e-02 -3.53771092e-01 -2.19637700e-01
 -8.50947921e-02  2.48209171e-01  1.44250148e-01 -1.11166843e-01
 -1.79112365e-01 -1.86511656e-01  4.13792981e-01  5.70879656e+01
 -1.12819673e+00  1.21134919e+00 -2.38740356e-01 -4.35140711e-01
  2.49660518e-01 -1.78845045e-01 -1.37894462e+00  6.20943546e-01
  1.27726830e+00 -4.37360604e-02  2.81971419e-01  2.44200797e-01
 -6.74010633e-02 -2.58111137e-01 -2.19657496e-01 -2.17908354e-01
  7.13880351e-02 -8.91208891e-01 -3.54825250e-01  1.65753817e-01
 -5.27235938e-03  4.08985231e-01 -9.88826594e-02 -3.51798067e-01
  1.44240940e+01  2.50844660e-01 -5.28979594e-01  2.18369224e-01
  2.25128848e-01  1.88515454e-01 -3.67316395e-01 -3.57641495e-01
 -2.47554853e-02 -1.33285712e-01  4.70482026e-01 

# Zadanie 3

In [19]:
TAU = lambda K: K / np.sqrt(2 * D)
TAU0 = lambda K: K / np.sqrt(2 * np.sqrt(D))

params = [
    # 1. Konfiguracja bazowa – mała populacja, umiarkowana mutacja
    {
        'mu': 10,
        'lam': 30,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 2. Większa liczba potomków, mocniejsza mutacja – większa eksploracja
    {
        'mu': 10,
        'lam': 70,
        'tau': TAU(0.2),
        'tau0': TAU0(0.2),
    },
    # 3. Duża populacja rodziców i potomków, umiarkowana mutacja – bardziej „stabilne” szukanie
    {
        'mu': 30,
        'lam': 200,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 4. Mała populacja, słaba mutacja – bardziej eksploatacja lokalna
    {
        'mu': 5,
        'lam': 25,
        'tau': TAU(0.05),
        'tau0': TAU0(0.05),
    },
]

# parametry
D = 100          # oryginalny wymiar F
H = 10 * D       # nowy wymiar G

A = np.random.uniform(0, 1, size=(D, H))
b = np.random.uniform(0, 1, size=D)

def G(y):
    return sphere(A @ y + b)

for p in params:
    # ES(μ + λ)
    best_plus = ES_plus(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=G,
        n=H,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ+λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_plus}\n"
        f"Wartość, którą osiąga: {-sphere(best_plus)}\n"
    )

    # ES(μ, λ)
    best_comma = ES_comma(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=G,
        n=H,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ,λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_comma}\n"
        f"Wartość, którą osiąga: {-sphere(best_comma)}\n"
    )

[ES(μ+λ)] mu=10, λ=30
Najlepszy osobnik: [ 7.87154357e+00 -8.12573314e+00 -4.14300106e+00  7.21446114e+00
 -8.77299239e+00 -2.71758779e+00  6.76859011e+00  1.44901279e+00
  5.80776713e+00  1.03746334e+00 -3.64767444e+00  7.09621565e+00
  3.66503841e+00 -9.55002506e+00 -5.15712676e+00 -2.78341361e+00
  3.53591483e+00  6.34304513e+00  6.71210046e+00 -7.20474708e+00
  9.69953036e+00 -1.89761693e+00 -4.91195802e+00 -1.87655812e+00
 -1.93929494e+00 -9.51725793e+00  3.38183329e+00  8.29506742e+00
 -8.23646868e+00  2.89151754e+00  8.37156102e+00  4.74582629e+00
 -7.81875593e-01  7.03669716e+00  5.37932714e+00  2.61559080e+00
 -3.43786439e+00  7.86747564e+00 -3.13596029e-01 -2.26348591e+00
  7.84038285e+00 -6.93485062e+00 -9.73437223e+00 -6.11413585e+00
 -5.66296082e-01 -5.07741724e+00 -5.63760714e+00 -3.31769752e+00
  8.97861916e+00 -6.64953379e+00  9.85358950e+00 -8.93476964e+00
 -6.02426042e+00  6.43846756e+00  8.81087108e+00  8.63903139e+00
 -4.25628577e+00 -1.17663685e+00  2.21509480e+00 

In [20]:
TAU = lambda K: K / np.sqrt(2 * D)
TAU0 = lambda K: K / np.sqrt(2 * np.sqrt(D))

params = [
    # 1. Konfiguracja bazowa – mała populacja, umiarkowana mutacja
    {
        'mu': 10,
        'lam': 30,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 2. Większa liczba potomków, mocniejsza mutacja – większa eksploracja
    {
        'mu': 10,
        'lam': 70,
        'tau': TAU(0.2),
        'tau0': TAU0(0.2),
    },
    # 3. Duża populacja rodziców i potomków, umiarkowana mutacja – bardziej „stabilne” szukanie
    {
        'mu': 30,
        'lam': 200,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 4. Mała populacja, słaba mutacja – bardziej eksploatacja lokalna
    {
        'mu': 5,
        'lam': 25,
        'tau': TAU(0.05),
        'tau0': TAU0(0.05),
    },
]

# parametry
D = 100          # oryginalny wymiar F
H = 10 * D       # nowy wymiar G

A = np.random.uniform(0, 1, size=(D, H))
b = np.random.uniform(0, 1, size=D)

def G(y):
    return dixon_price(A @ y + b)

for p in params:
    # ES(μ + λ)
    best_plus = ES_plus(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=G,
        n=H,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ+λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_plus}\n"
        f"Wartość, którą osiąga: {-dixon_price(best_plus)}\n"
    )

    # ES(μ, λ)
    best_comma = ES_comma(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=G,
        n=H,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ,λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_comma}\n"
        f"Wartość, którą osiąga: {-dixon_price(best_comma)}\n"
    )

[ES(μ+λ)] mu=10, λ=30
Najlepszy osobnik: [ 7.23217389  9.75728946  1.8643144  -1.69673481 -9.18260336  9.34395636
 -4.5871965  -5.47140309 -5.35431744 -8.29450847 -7.0294099  -9.55680023
  1.55348313 -7.50565837 -7.74646863  1.06565944  0.33668541 -4.75062517
  9.25050652 -6.68349138  9.33789215  2.0977288   1.79808273 -9.22450243
 -4.82819838 -4.99329255 -8.91522215  0.56452862  2.05106126  1.32548205
 -3.40341524  6.39507956 -3.54357159  9.04676177  5.34540087 -6.92071201
  7.9035449  -3.15868566 -4.82229933  7.40615724 -9.00536371  5.05002095
 -7.10059056 -7.33447902 -2.53496726 -7.82443334 -5.5594499   6.84356675
  6.7017928   6.54378308  2.95673435 -7.12612702  8.9329332   7.85501209
  2.52603071 -3.5292549  -8.97605149 -9.06743468 -0.11818407 -4.16927287
  0.35817552 -9.28547612 -5.1144703  -9.81941372 -2.23934344  4.86247315
 -6.09354217  1.84995573 -7.30874385 -6.45979432 -0.283434    1.10215849
  6.91333625 -0.72391837  2.28898404 -2.75953742 -1.16300839 -4.38635199
 -9.999973

# Problemy z ograniczeniami

In [21]:
TAU = lambda K: K / np.sqrt(2 * D)
TAU0 = lambda K: K / np.sqrt(2 * np.sqrt(D))

params = [
    # 1. Konfiguracja bazowa – mała populacja, umiarkowana mutacja
    {
        'mu': 10,
        'lam': 30,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 2. Większa liczba potomków, mocniejsza mutacja – większa eksploracja
    {
        'mu': 10,
        'lam': 70,
        'tau': TAU(0.2),
        'tau0': TAU0(0.2),
    },
    # 3. Duża populacja rodziców i potomków, umiarkowana mutacja – bardziej „stabilne” szukanie
    {
        'mu': 30,
        'lam': 200,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 4. Mała populacja, słaba mutacja – bardziej eksploatacja lokalna
    {
        'mu': 5,
        'lam': 25,
        'tau': TAU(0.05),
        'tau0': TAU0(0.05),
    },
]

# parametry
D = 100          # oryginalny wymiar F
H = 10 * D       # nowy wymiar G

A = np.random.uniform(0, 1, size=(D, H))
b = np.random.uniform(0, 1, size=D)

def G(y):
    return schwefel(A @ y + b)

for p in params:
    # ES(μ + λ)
    best_plus = ES_plus(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=G,
        n=H,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ+λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_plus}\n"
        f"Wartość, którą osiąga: {-schwefel(best_plus)}\n"
    )

    # ES(μ, λ)
    best_comma = ES_comma(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=G,
        n=H,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ,λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_comma}\n"
        f"Wartość, którą osiąga: {-schwefel(best_comma)}\n"
    )

[ES(μ+λ)] mu=10, λ=30
Najlepszy osobnik: [ 3.01392199 -3.12457721 -1.78770935 -8.79729067 -4.22818954  7.60196926
  0.73383731  9.61445566 -4.54367909  6.72275268 -6.23038175  2.13117683
 -6.23259474  6.9295158   3.78991686 -8.37002619  3.62147538 -9.96461562
 -2.48319208  1.77148447 -3.87990684 -7.76112577 -2.5943346  -9.9081187
 -6.13303492  5.82112003  5.32401007  9.26947382 -8.58183489 -7.02276784
  1.48590342  6.48065321 -0.37580756  7.58313018  5.2441588  -5.20581059
  3.41905811  4.31045839  2.07666736  2.33606763  5.83180112 -8.7662082
 -8.23672255  9.59966445  3.16541221  5.71918985  6.62880463 -9.26158978
 -8.94743459 -3.33661498 -2.41013627 -8.98793309  0.06894211 -6.67927837
  0.91566161  3.56571408  1.36965285  1.75794466 -6.02661294  1.88809726
  6.89597544  5.56900732 -3.44305732 -8.14926599 -0.42361199  8.69912502
 -5.07388183  3.20840568  1.28177529 -8.89878349 -5.34040241 -7.76793468
  7.2780568   5.50191852  0.42082812  3.02314457 -9.65454938 -0.11162535
 -1.99114389

In [22]:
TAU = lambda K: K / np.sqrt(2 * D)
TAU0 = lambda K: K / np.sqrt(2 * np.sqrt(D))

params = [
    # 1. Konfiguracja bazowa – mała populacja, umiarkowana mutacja
    {
        'mu': 10,
        'lam': 30,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 2. Większa liczba potomków, mocniejsza mutacja – większa eksploracja
    {
        'mu': 10,
        'lam': 70,
        'tau': TAU(0.2),
        'tau0': TAU0(0.2),
    },
    # 3. Duża populacja rodziców i potomków, umiarkowana mutacja – bardziej „stabilne” szukanie
    {
        'mu': 30,
        'lam': 200,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 4. Mała populacja, słaba mutacja – bardziej eksploatacja lokalna
    {
        'mu': 5,
        'lam': 25,
        'tau': TAU(0.05),
        'tau0': TAU0(0.05),
    },
]

# parametry
D = 100          # oryginalny wymiar F
H = 10 * D       # nowy wymiar G

A = np.random.uniform(0, 1, size=(D, H))
b = np.random.uniform(0, 1, size=D)

def G(y):
    return rastrigin(A @ y + b)

for p in params:
    # ES(μ + λ)
    best_plus = ES_plus(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=G,
        n=H,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ+λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_plus}\n"
        f"Wartość, którą osiąga: {-rastrigin(best_plus)}\n"
    )

    # ES(μ, λ)
    best_comma = ES_comma(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=G,
        n=H,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ,λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_comma}\n"
        f"Wartość, którą osiąga: {-rastrigin(best_comma)}\n"
    )

[ES(μ+λ)] mu=10, λ=30
Najlepszy osobnik: [ 9.51172989e+00 -1.99172288e+00 -6.88569357e+00 -2.38704167e+00
 -7.00712872e+00 -3.22283586e+00  4.56067898e+00 -2.82017448e+00
 -3.18266514e+00 -7.10672319e+00 -5.04081565e+00  3.73891568e+00
 -9.94907395e+00  6.99330545e+00  6.95983554e+00 -9.33309745e+00
  3.34388757e+00 -3.81632755e+00  7.63439223e+00  7.06628004e-01
  2.89990416e+00  4.07850292e+00 -9.73968631e+00  8.27956789e-01
  1.19175670e+00 -8.63497012e+00  2.73086974e+00 -5.97273711e+00
  5.43720263e+00  6.34335286e+00 -5.03319622e+00  3.48283346e+00
 -6.62906892e+00 -2.33666829e+00  7.47571666e+00  3.05341484e+00
  2.74932701e+00  5.76289632e+00  5.79979663e+00  4.81551218e+00
  9.68406701e-01  4.37055816e+00  2.26767424e+00 -3.40446032e+00
 -9.70118418e+00 -2.83281560e+00 -4.14734583e+00  2.99441689e+00
 -3.48756042e+00  5.49851216e+00  2.71727063e+00 -4.59434948e-01
 -6.66593419e+00 -5.50078796e+00  8.82267011e-01 -5.05094476e-01
  1.03403953e+00  1.09703942e+00  3.11432294e-01 

In [23]:
TAU = lambda K: K / np.sqrt(2 * D)
TAU0 = lambda K: K / np.sqrt(2 * np.sqrt(D))

params = [
    # 1. Konfiguracja bazowa – mała populacja, umiarkowana mutacja
    {
        'mu': 10,
        'lam': 30,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 2. Większa liczba potomków, mocniejsza mutacja – większa eksploracja
    {
        'mu': 10,
        'lam': 70,
        'tau': TAU(0.2),
        'tau0': TAU0(0.2),
    },
    # 3. Duża populacja rodziców i potomków, umiarkowana mutacja – bardziej „stabilne” szukanie
    {
        'mu': 30,
        'lam': 200,
        'tau': TAU(0.1),
        'tau0': TAU0(0.1),
    },
    # 4. Mała populacja, słaba mutacja – bardziej eksploatacja lokalna
    {
        'mu': 5,
        'lam': 25,
        'tau': TAU(0.05),
        'tau0': TAU0(0.05),
    },
]

# parametry
D = 100          # oryginalny wymiar F
H = 10 * D       # nowy wymiar G

A = np.random.uniform(0, 1, size=(D, H))
b = np.random.uniform(0, 1, size=D)

def G(y):
    return griewank(A @ y + b)

for p in params:
    # ES(μ + λ)
    best_plus = ES_plus(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=G,
        n=H,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ+λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_plus}\n"
        f"Wartość, którą osiąga: {-griewank(best_plus)}\n"
    )

    # ES(μ, λ)
    best_comma = ES_comma(
        mu=p['mu'],
        lam=p['lam'],
        tau=p['tau'],
        tau0=p['tau0'],
        F=G,
        n=H,
        x_min=-10.0,
        x_max=10.0
    )['x']
    print(
        f"[ES(μ,λ)] mu={p['mu']}, λ={p['lam']}\n"
        f"Najlepszy osobnik: {best_comma}\n"
        f"Wartość, którą osiąga: {-griewank(best_comma)}\n"
    )

[ES(μ+λ)] mu=10, λ=30
Najlepszy osobnik: [ 1.51772045e-01  4.23998179e+00  1.03595367e+00  3.71424282e+00
  1.51900442e+00  1.95564990e+00 -8.47870555e+00 -9.19721196e+00
 -3.12256219e+00  9.99740111e+00  6.42698381e-01  9.42247051e+00
 -7.71274003e+00 -4.50466737e+00 -7.77180177e+00 -7.49416851e+00
  6.28389079e+00  8.98920519e+00  9.08513054e+00 -4.17031761e+00
  4.44555779e+00 -2.55540659e+00  5.49771656e+00  6.89070654e+00
  3.90427869e+00  7.15884159e-01  3.34350384e+00 -4.80515290e+00
  5.59959587e+00 -5.14375717e+00 -8.80338740e+00  2.25853858e+00
  6.32409393e+00 -3.62822676e+00 -8.73046299e+00  5.18650261e+00
 -6.09250351e+00  4.75778299e+00  9.90750475e+00  2.69065390e+00
 -3.62876795e+00  5.80859672e-01  5.96723157e+00  7.67114737e-01
 -2.34989828e+00 -4.09257923e+00  6.87976218e+00 -4.54137596e+00
  2.94872497e+00  8.79371556e+00  6.10201778e+00  3.26216014e+00
 -4.57296036e+00 -8.70848704e+00  5.50256567e+00 -2.47865865e+00
  5.25694373e+00 -5.74645870e+00  7.42906741e+00 

In [24]:
import numpy as np
import copy

In [25]:
# Tworzy losowego osobnika
def random_individual(n: int, x_min: float = -5.0, x_max: float = 5.0):
    return {
        'x': np.random.uniform(x_min, x_max, size=n),
        'o': np.random.uniform(0.1, 1.0, size=n)   # sigma
    }

# Tworzy populację
def random_population(mu: int, n: int, x_min: float = -5.0, x_max: float = 5.0):
    return np.array([random_individual(n, x_min, x_max) for _ in range(mu)], dtype=object)

# Ocena populacji
def population_evaluation(P, F, G):
    if G:
        return np.array([F(ind['x']) - G(ind['x']) for ind in P])
    return np.array([F(ind['x']) for ind in P])

# Selekcja rodziców (z powtórzeniami!)
def parent_selection(P, lam: int):
    idx = np.random.choice(len(P), lam, replace=True)
    return P[idx]

# Mutacja gaussowska
def mutation(Pc, tau, tau0, low, high):
    lam = len(Pc)
    n = len(Pc[0]['x'])

    for i in range(lam):
        indiv = Pc[i]
        x = indiv['x']
        o = indiv['o']

        # Mutacja parametrów strategii
        N0 = np.random.normal()
        Ni = np.random.normal(size=n)

        o_new = o * np.exp(tau0 * N0 + tau * Ni)

        # Mutacja wartości x
        x_new = x + o_new * np.random.normal(size=n)
        x_new = np.clip(x_new, low, high)

        indiv['x'] = x_new
        indiv['o'] = o_new

    return Pc

def replacement_plus(P, Pc, mu, F, G):
    combined = np.array([copy.deepcopy(ind) for ind in list(P) + list(Pc)], dtype=object)
    fitness = population_evaluation(combined, F, G)
    idx = np.argsort(fitness)[-mu:]   # najlepsze mu
    return combined[idx]

# Replacement: ES(mu, lambda)
def replacement_comma(Pc, mu, F, G):
    combined = np.array([copy.deepcopy(ind) for ind in list(Pc)], dtype=object)
    fitness = population_evaluation(combined, F, G)
    idx = np.argsort(fitness)[-mu:]   # najlepsze mu
    return combined[idx]

In [26]:
ITER = 10000

# Strategia ewolucyjna ES(mu + lambda)
def ES_plus(mu, lam, tau, tau0, F, G, n, ITER=ITER, x_min: float = -5.0, x_max: float = 5.0):
    P = random_population(mu, n, x_min, x_max)

    for _ in range(ITER):
        Pc = parent_selection(P, lam)
        Pc = mutation(Pc, tau, tau0, x_min, x_max)
        P = replacement_plus(P, Pc, mu, F, G)

    fitness = population_evaluation(P, F, G)
    return P[np.argmax(fitness)]

# Strategia ewolucyjna ES(mu, lambda)
def ES_comma(mu, lam, tau, tau0, F, G, n, ITER=ITER, x_min: float = -5.0, x_max: float = 5.0):
    P = random_population(mu, n, x_min, x_max)

    for _ in range(ITER):
        Pc = parent_selection(P, lam)
        Pc = mutation(Pc, tau, tau0, x_min, x_max)
        P = replacement_comma(Pc, mu, F, G)

    fitness = population_evaluation(P, F, G)
    return P[np.argmax(fitness)]

In [27]:
# --------------------------
# Funkcja celu G2 (maksymalizacja)
# --------------------------
def g2_objective(x):
    x = np.array(x)
    n = len(x)

    numerator = np.sum(np.cos(x)**4) - 2 * np.prod(np.cos(x)**2)
    denominator = np.sqrt(np.sum((np.arange(1, n+1) * x**2)))

    return numerator / denominator


# --------------------------
# Ograniczenia (<= 0)
# --------------------------

def g2_constraint_1(x):
    # g1(x) = Π xi  + 0.75 <= 0
    return -np.prod(x) + 0.75


def g2_constraint_2(x):
    # g2(x) = Σ xi - 7.5n <= 0
    x = np.array(x)
    n = len(x)
    return np.sum(x) - 7.5 * n

def g_penalty(x):
    penalty = 0

    c1 = g2_constraint_1(x)
    c2 = g2_constraint_2(x)

    if c1 > 0:
        penalty += 1000000

    if c2 > 0:
        penalty += 1000000

    return penalty

D = 20
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=g2_objective,  G=g_penalty, n=D, x_min=0.0, x_max=10.0)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {g2_objective(best)}\n')
print(g2_constraint_1(best))
print(g2_constraint_2(best))

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=g2_objective, G=g_penalty, n=D, x_min=0.0, x_max=10.0)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {g2_objective(best)}')
print(g2_constraint_1(best))
print(g2_constraint_2(best))


Najlepszy osobnik: [3.13842936e+00 3.13598030e+00 3.13391563e+00 9.39255758e+00
 3.04743260e-03 3.12545694e+00 6.24547283e+00 9.35965920e+00
 1.81882631e-03 2.10488189e-03 3.11169917e+00 9.33300865e+00
 6.21305940e+00 6.20723377e+00 9.30100127e+00 3.09844457e+00
 2.56918574e-03 3.08055284e+00 3.09068738e+00 1.54272067e+00]
Wartość, którą osiąga: 0.2550874721785946

-0.0021624598664310435
-67.48058010420094
Najlepszy osobnik: [3.13856091e+00 7.83835742e+00 9.80433941e-05 9.38839435e+00
 6.25300927e+00 3.12351966e+00 6.24085535e+00 6.23482220e+00
 3.11448315e+00 6.22269968e+00 3.10849980e+00 3.10559470e+00
 3.10255007e+00 6.19829700e+00 3.09657211e+00 3.09380557e-04
 1.43746067e-04 3.08763425e+00 3.08466635e+00 9.23528564e+00]
Wartość, którą osiąga: 0.2699867097646912
-2.0238594762522766e-05
-64.42564692434183


In [28]:
# Funkcja celu G3 (maksymalizacja)
def g6_objective(x):
    x = np.array(x)
    n = len(x)
    return -(np.pow(x[0] - 10, 3) + np.pow(x[1] + 20, 3))

# Ograniczenie równościowe h1(x) = 0
def g6_constraint_1(x):
    x = np.array(x)
    return np.pow(x[0] - 5, 2) + np.pow(x[1] - 5, 2) + 100

def g6_constraint_2(x):
    x = np.array(x)
    return np.pow(x[0] - 5, 2) + np.pow(x[1] - 5, 2) + 82.81

def g_penalty(x):
    penalty = 0

    c1 = g6_constraint_1(x)

    if c1 > 0:
        penalty = 1000000

    return penalty

D = 2
K = 0.1
TAU = K / np.sqrt(2 * D)
TAU0 =   K / np.sqrt(2 * np.sqrt(D))

best = ES_plus(mu=10, lam=30, tau=TAU, tau0=TAU0, F=g6_objective,  G=g_penalty, n=D, x_min=0.0, x_max=100.0)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-g6_objective(best)}\n')
print(g6_constraint_1(best))

best = ES_comma(mu=10, lam=30, tau=TAU, tau0=TAU0, F=g6_objective, G=g_penalty, n=D, x_min=0.0, x_max=100.0)['x']
print(f'Najlepszy osobnik: {best}\nWartość, którą osiąga: {-g6_objective(best)}')
print(g6_constraint_1(best))


Najlepszy osobnik: [4.19483682e-14 3.58041799e-15]
Wartość, którą osiąga: 7000.000000000017

149.99999999999955
Najlepszy osobnik: [1.22530338e-13 1.46927297e-14]
Wartość, którą osiąga: 7000.000000000054
149.99999999999864
